### Data Cleaning

In [1]:
# load necessary packages
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

#### Vital Statistics Mortality Data

In [2]:
# load the datasets: 2003 - 2016
list_mortality_datasets = [x for x in range(2003, 2016)]
death_df = pd.DataFrame()
for year in list_mortality_datasets:
    death_data = pd.read_csv(
        f"https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%20{year}.txt",
        delimiter="\t",
    )
    death_data["year"] = year
    death_df = pd.concat([death_df, death_data], ignore_index=True)

In [3]:
# year: 2007-2012,
# states: Treatment - Florida, Control - California, New Jersey, Oregon
# remove Year and Year code (redundant)
death_df = death_df.drop(columns=["Year", "Year Code"])
# Deaths
death_df["Deaths"] = pd.to_numeric(death_df["Deaths"], errors="coerce").astype(float)

In [4]:
# adjust FIPS code
death_df["County Code"] = (
    death_df["County Code"]
    .astype(str)
    .str.strip()  # remove space
    .str.replace(r"\.0$", "", regex=True)  # remove .0
    .str.zfill(5)  # 5 digits
)

# Check the transoformation of County Code
death_df[death_df["County Code"].str.len() != 5]["County Code"].unique()

# check empty values and potentail duplicated values
death_df["County Code"].isna().sum()

0

In [5]:
# year: 2007 - 2012
death_df_1 = death_df[(death_df["year"] >= 2007) & (death_df["year"] <= 2012)]

# 4 states: Florida (12), California(06), New Jersey(34), Oregon(41)
target_states = ["12", "06", "34", "41"]

death_df_1 = death_df_1[death_df_1["County Code"].str.startswith(tuple(target_states))]

death_df_1.head(10)

,Notes,County,County Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,year
16983,NaN,"Alameda County, CA",06001,Drug poisonings (overdose) Unintentional (X40-...,D1,163.0,2007
16984,NaN,"Alameda County, CA",06001,Drug poisonings (overdose) Suicide (X60-X64),D2,22.0,2007
16985,NaN,"Alameda County, CA",06001,"Alcohol poisonings (overdose) (X45, X65, Y15)",A1,13.0,2007
16986,NaN,"Alameda County, CA",06001,All other alcohol-induced causes,A9,129.0,2007
16987,NaN,"Alameda County, CA",06001,All other non-drug and non-alcohol causes,O9,8989.0,2007
16988,NaN,"Amador County, CA",06005,All other non-drug and non-alcohol causes,O9,376.0,2007
16989,NaN,"Butte County, CA",06007,Drug poisonings (overdose) Unintentional (X40-...,D1,58.0,2007
16990,NaN,"Butte County, CA",06007,All other alcohol-induced causes,A9,22.0,2007
16991,NaN,"Butte County, CA",06007,All other non-drug and non-alcohol causes,O9,2175.0,2007
16992,NaN,"Calaveras County, CA",06009,All other non-drug and non-alcohol causes,O9,397.0,2007


#### Opioid Prescriptions Data

In [6]:
# import the opioid data
opioids_cy = pd.read_parquet(
    "https://github.com/nickeubank/ids540_opioid_data/raw/refs/heads/main/ids590_opioids_by_drug_county_year.parquet"
)

# import fips code
op_fips = pd.read_csv(
    "https://raw.githubusercontent.com/wpinvestigative/arcos-api/refs/heads/master/data/county_fips.csv"
)

In [7]:
# Combine opioids with FIPS code
# change the name in op_fips data
op_fips = op_fips.rename(
    columns={"BUYER_STATE": "buyer_state", "BUYER_COUNTY": "buyer_county"}
)

# adjust the fips code
op_fips["countyfips"] = op_fips["countyfips"].astype(str).str.zfill(5)
op_fips.head(10)

,buyer_county,buyer_state,countyfips
0,AUTAUGA,AL,01001
1,BALDWIN,AL,01003
2,BARBOUR,AL,01005
3,BIBB,AL,01007
4,BLOUNT,AL,01009
5,BULLOCK,AL,01011
6,BUTLER,AL,01013
7,CALHOUN,AL,01015
8,CHAMBERS,AL,01017
9,CHEROKEE,AL,01019


In [8]:
# Combine two datasets
opioids_df = pd.merge(
    opioids_cy, op_fips, how="outer", on=["buyer_state", "buyer_county"]
)

# have checked the matching: indicator=True
# opioids_df[opioids_df._merge != "both"], all rows match successfully

In [9]:
# year: 2007 - 2012
opioids_df_1 = opioids_df[(opioids_df["year"] >= 2007) & (opioids_df["year"] <= 2012)]
# integer
opioids_df_1["year"] = opioids_df_1["year"].astype(int)


# 4 states: FL, CA, NJ, OR
opioids_df_1 = opioids_df_1[opioids_df_1["buyer_state"].isin(["FL", "CA", "NJ", "OR"])]

# check the 4 states
opioids_df_1.groupby("buyer_state")["buyer_county"].nunique()

buyer_state
CA    57
FL    67
NJ    21
OR    34
Name: buyer_county, dtype: int64

> In total, 4 states should have 182 states, and mortality data does have 182 states, whereas opioids_df_1 only has 167 states. CA - 58, FL - 67, NJ - 21, OR - 26

In [10]:
opioids_df_1.head(10)

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm,countyfips
32697,CA,ALAMEDA,2007,BUPRENORPHINE,30.0,951.545738,06001
32698,CA,ALAMEDA,2007,BUPRENORPHINE,75.0,0.042071,06001
32699,CA,ALAMEDA,2007,CODEINE,0.15,209552.490726,06001
32700,CA,ALAMEDA,2007,DIHYDROCODEINE,0.25,15.176001,06001
32701,CA,ALAMEDA,2007,FENTANYL,100.0,1852.491745,06001
32702,CA,ALAMEDA,2007,FENTANYL,130.0,95.805201,06001
32703,CA,ALAMEDA,2007,HYDROCODONE,1.0,343541.935046,06001
32704,CA,ALAMEDA,2007,HYDROMORPHONE,4.0,5551.601937,06001
32705,CA,ALAMEDA,2007,LEVORPHANOL,11.0,86.4756,06001
32706,CA,ALAMEDA,2007,MEPERIDINE,0.1,5524.634702,06001


> After a series of checking , for opioids_df_1, most counties missing here are counties with small populations. 

In [11]:
# Combine opioid and mortality data
death_df_1.head(10)

,Notes,County,County Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,year
16983,NaN,"Alameda County, CA",06001,Drug poisonings (overdose) Unintentional (X40-...,D1,163.0,2007
16984,NaN,"Alameda County, CA",06001,Drug poisonings (overdose) Suicide (X60-X64),D2,22.0,2007
16985,NaN,"Alameda County, CA",06001,"Alcohol poisonings (overdose) (X45, X65, Y15)",A1,13.0,2007
16986,NaN,"Alameda County, CA",06001,All other alcohol-induced causes,A9,129.0,2007
16987,NaN,"Alameda County, CA",06001,All other non-drug and non-alcohol causes,O9,8989.0,2007
16988,NaN,"Amador County, CA",06005,All other non-drug and non-alcohol causes,O9,376.0,2007
16989,NaN,"Butte County, CA",06007,Drug poisonings (overdose) Unintentional (X40-...,D1,58.0,2007
16990,NaN,"Butte County, CA",06007,All other alcohol-induced causes,A9,22.0,2007
16991,NaN,"Butte County, CA",06007,All other non-drug and non-alcohol causes,O9,2175.0,2007
16992,NaN,"Calaveras County, CA",06009,All other non-drug and non-alcohol causes,O9,397.0,2007


In [12]:
death_df_1["Drug/Alcohol Induced Cause"].value_counts()

Drug/Alcohol Induced Cause
All other non-drug and non-alcohol causes             1085
All other alcohol-induced causes                       631
Drug poisonings (overdose) Unintentional (X40-X44)     581
Drug poisonings (overdose) Suicide (X60-X64)           216
Alcohol poisonings (overdose) (X45, X65, Y15)           68
All other drug-induced causes                           60
Drug poisonings (overdose) Undetermined (Y10-Y14)       42
Name: count, dtype: int64

In [13]:
# define the specific overdoes category we want to keep
drug_induced_death = ["Drug poisonings (overdose) Unintentional (X40-X44)"]

# filter the death dataset to include only unintentional drug overdoes deathes (X40-X44) as required
death_df_overdose = death_df_1[
    death_df_1["Drug/Alcohol Induced Cause"].isin(drug_induced_death)
]

In [14]:
# calculate the total deaths
death_overdose = (
    death_df_overdose.groupby(["County Code", "year"], as_index=False)["Deaths"]
    .sum()
    .rename(columns={"Deaths": "overdose_deaths"})
)
death_overdose.head(10)

,County Code,year,overdose_deaths
0,06001,2007,163.0
1,06001,2008,135.0
2,06001,2009,119.0
3,06001,2010,106.0
4,06001,2011,139.0
5,06001,2012,115.0
6,06005,2009,13.0
7,06007,2007,58.0
8,06007,2008,58.0
9,06007,2009,49.0


In [15]:
opioids_df_1.head(10)

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm,countyfips
32697,CA,ALAMEDA,2007,BUPRENORPHINE,30.0,951.545738,06001
32698,CA,ALAMEDA,2007,BUPRENORPHINE,75.0,0.042071,06001
32699,CA,ALAMEDA,2007,CODEINE,0.15,209552.490726,06001
32700,CA,ALAMEDA,2007,DIHYDROCODEINE,0.25,15.176001,06001
32701,CA,ALAMEDA,2007,FENTANYL,100.0,1852.491745,06001
32702,CA,ALAMEDA,2007,FENTANYL,130.0,95.805201,06001
32703,CA,ALAMEDA,2007,HYDROCODONE,1.0,343541.935046,06001
32704,CA,ALAMEDA,2007,HYDROMORPHONE,4.0,5551.601937,06001
32705,CA,ALAMEDA,2007,LEVORPHANOL,11.0,86.4756,06001
32706,CA,ALAMEDA,2007,MEPERIDINE,0.1,5524.634702,06001


In [16]:
# Multiply the drug’s base weight by its conversion factor to compute each record’s morphine milligram equivalent (MME).
opioids_df_1["mme"] = (
    opioids_df_1["calc_base_wt_in_gm"] * opioids_df_1["mme_conversion_factor"]
)
opioids_df_1.head(10)

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm,countyfips,mme
32697,CA,ALAMEDA,2007,BUPRENORPHINE,30.0,951.545738,06001,28546.37214
32698,CA,ALAMEDA,2007,BUPRENORPHINE,75.0,0.042071,06001,3.155355
32699,CA,ALAMEDA,2007,CODEINE,0.15,209552.490726,06001,31432.873609
32700,CA,ALAMEDA,2007,DIHYDROCODEINE,0.25,15.176001,06001,3.794
32701,CA,ALAMEDA,2007,FENTANYL,100.0,1852.491745,06001,185249.17455
32702,CA,ALAMEDA,2007,FENTANYL,130.0,95.805201,06001,12454.676078
32703,CA,ALAMEDA,2007,HYDROCODONE,1.0,343541.935046,06001,343541.935046
32704,CA,ALAMEDA,2007,HYDROMORPHONE,4.0,5551.601937,06001,22206.407747
32705,CA,ALAMEDA,2007,LEVORPHANOL,11.0,86.4756,06001,951.2316
32706,CA,ALAMEDA,2007,MEPERIDINE,0.1,5524.634702,06001,552.46347


In [17]:
# aggregate opioid shipments to the county-year level
# i.e. sum total grams and total MME for each county-year
opioids_agg = opioids_df_1.groupby(["countyfips", "year"], as_index=False).agg(
    total_grams=("calc_base_wt_in_gm", "sum"), total_mme=("mme", "sum")
)
opioids_agg.head(10)

,countyfips,year,total_grams,total_mme
0,06001,2007,825601.233034,1052558.500894
1,06001,2008,847083.303522,1117798.930286
2,06001,2009,875071.886798,1199622.098446
3,06001,2010,870211.603741,1247010.639905
4,06001,2011,896433.100448,1296731.400855
5,06001,2012,900341.807573,1313075.53561
6,06005,2007,28486.513615,50147.191799
7,06005,2008,31402.138263,56498.072585
8,06005,2009,34543.746528,63986.757334
9,06005,2010,34834.035017,66070.709978


#### NHGIS dataset for control variables

In [18]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

nhgis = pd.read_csv("nhgis0001_ts_nominal_county.csv")
nhgis.head()

,GISJOIN,STATE,STATEFP,STATENH,COUNTY,COUNTYFP,COUNTYNH,AV0AA115,AV0AA115M,AV0AA135,...,BS7AB135,BS7AB135M,BS7AC115,BS7AC115M,BS7AC135,BS7AC135M,BS7AD115,BS7AD115M,BS7AD135,BS7AD135M
0,G0100010,Alabama,1,10,Autauga County,1,10,53944,0,54907,...,1099,313,1757,296,1805,316,16094,814,15903,864
1,G0100030,Alabama,1,10,Baldwin County,3,30,179523,0,187114,...,3571,446,8204,793,8996,796,55072,1824,55940,1831
2,G0100050,Alabama,1,10,Barbour County,5,50,27546,0,27321,...,1063,165,1329,204,1423,211,5693,433,5388,409
3,G0100070,Alabama,1,10,Bibb County,7,70,22746,0,22754,...,472,173,1157,244,1379,252,4985,548,4539,475
4,G0100090,Alabama,1,10,Blount County,9,90,57140,0,57623,...,1159,207,2897,379,2753,359,15149,806,15532,840


In [19]:
# define variable prefixes we need (according to the code book)
needed_prefixes = [
    "AV0",  # population
    "AV1",  # sex
    "B57",  # age (18 groups)
    "B18",  # race (5 categories)
    "BW7",  # educational attainment (5 groups)
    "BS7",  # household income
]

In [20]:
# map raw nhgis variable codes to clear variable names for 2009 and 2011
# collapse time period: 2007-2011 -> 2009; 2009-2013 -> 2011
rename_map = {
    # POPULATION
    "AV0AA115": "population_2009",
    "AV0AA135": "population_2011",
    # SEX
    "AV1AA115": "male_2009",
    "AV1AA135": "male_2011",
    "AV1AB115": "female_2009",
    "AV1AB135": "female_2011",
    # AGE (18 groups)
    "B57AA115": "under_5__years_2009",
    "B57AA135": "under_5__years_2011",
    "B57AB115": "5_to_9_years_2009",
    "B57AB135": "5_to_9_years_2011",
    "B57AC115": "10_to_14_years_2009",
    "B57AC135": "10_to_14_years_2011",
    "B57AD115": "15_to_17_years_2009",
    "B57AD135": "15_to_17_years_2011",
    "B57AE115": "18_to_19_years_2009",
    "B57AE135": "18_to_19_years_2011",
    "B57AF115": "20_years_2009",
    "B57AF135": "20_years_2011",
    "B57AG115": "21_years_2009",
    "B57AG135": "21_years_2011",
    "B57AH115": "22_to_24_years_2009",
    "B57AH135": "22_to_24_years_2011",
    "B57AI115": "25_to_29_years_2009",
    "B57AI135": "25_to_29_years_2011",
    "B57AJ115": "30_to_34_years_2009",
    "B57AJ135": "30_to_34_years_2011",
    "B57AK115": "35_to_44_years_2009",
    "B57AK135": "35_to_44_years_2011",
    "B57AL115": "45_to_54_years_2009",
    "B57AL135": "45_to_54_years_2011",
    "B57AM115": "55_to_59_years_2009",
    "B57AM135": "55_to_59_years_2011",
    "B57AN115": "60_to_61_years_2009",
    "B57AN135": "60_to_61_years_2011",
    "B57AO115": "62_to_64_years_2009",
    "B57AO135": "62_to_64_years_2011",
    "B57AP115": "65_to_74_years_2009",
    "B57AP135": "65_to_74_years_2011",
    "B57AQ115": "75_to_84_years_2009",
    "B57AQ135": "75_to_84_years_2011",
    "B57AR115": "85_years_and_over_2009",
    "B57AR135": "85_years_and_over_2011",
    # RACE (5 groups)
    "B18AA115": "White_2009",
    "B18AA135": "White_2011",
    "B18AB115": "Black_or_African_American_2009",
    "B18AB135": "Black_or_African_American_2011",
    "B18AC115": "American_Indian_and_Alaska_Native_2009",
    "B18AC135": "American_Indian_and_Alaska_Native_2011",
    "B18AD115": "Asian_and_Pacific_Islander_and_Other_Race_2009",
    "B18AD135": "Asian_and_Pacific_Islander_and_Other_Race_2011",
    "B18AE115": "Two_or_More_Races_2009",
    "B18AE135": "Two_or_More_Races_2011",
    # EDUCATIONAL ATTAINMENT (5 groups)
    "BW7AA115": "Less_than_9th_grade_2009",
    "BW7AA135": "Less_than_9th_grade_2011",
    "BW7AB115": "9th_to_11th_grade_2009",
    "BW7AB135": "9th_to_11th_grade_2011",
    "BW7AC115": "12th_grade_or_high_school_graduate_2009",
    "BW7AC135": "12th_grade_or_high_school_graduate_2011",
    "BW7AD115": "1_to_3_years_of_college_2009",
    "BW7AD135": "1_to_3_years_of_college_2011",
    "BW7AE115": "4_or_more_years_of_college_2009",
    "BW7AE135": "4_or_more_years_of_college_2011",
    # HOUSEHOLD INCOME (4 groups)
    "BS7AA115": "less_than_10,000_2009",
    "BS7AA135": "less_than_10,000_2011",
    "BS7AB115": "10,000_to_14,999_2009",
    "BS7AB135": "10,000_to_14,999_2011",
    "BS7AC115": "15,000_to_24,999_2009",
    "BS7AC135": "15,000_to_24,999_2011",
    "BS7AD115": "25,000_or_more_2009",
    "BS7AD135": "25,000_or_more_2011",
}

In [21]:
# select only geographic identifiers + the variables we need
geo_cols = ["GISJOIN", "STATE", "COUNTY", "COUNTYFP"]

data_cols = []
# loop through all columns in the raw nhgis dataset
for col in nhgis.columns:
    # to check whether the column belongs to any table we care about
    # (population, sex, age, race, education, income), identified by prefixes
    for prefix in needed_prefixes:
        if col.startswith(prefix):
            # if the column matches one of the prefixes, append it
            data_cols.append(col)
            break

In [22]:
# subset the dataset to keep only:
# (1) geographic identifiers (GISJOIN, STATE, COUNTY, COUNTYFP)
# (2) selected demographic/ACS variables we need above
nhgis_cleaned = nhgis[geo_cols + data_cols]

# rename raw nhgis variable codes to human-readable names using rename_map
nhgis_cleaned = nhgis_cleaned.rename(columns=rename_map)

# select all columns that are from 2009 ACS
cols_2009 = [col for col in nhgis_cleaned.columns if col.endswith("_2009")]
# select all columns that are from 2011 ACS
cols_2011 = [col for col in nhgis_cleaned.columns if col.endswith("_2011")]

In [23]:
nhgis_cleaned[cols_2009]

,population_2009,male_2009,female_2009,under_5__years_2009,5_to_9_years_2009,10_to_14_years_2009,15_to_17_years_2009,18_to_19_years_2009,20_years_2009,21_years_2009,...,Two_or_More_Races_2009,Less_than_9th_grade_2009,9th_to_11th_grade_2009,12th_grade_or_high_school_graduate_2009,1_to_3_years_of_college_2009,4_or_more_years_of_college_2009,"less_than_10,000_2009","10,000_to_14,999_2009","15,000_to_24,999_2009","25,000_or_more_2009"
0,53944,26174,27770,3568,4095,4209,2836,1648,615,428,...,835,1551,2555,14997,8000,7478,1304,843,1757,16094
1,179523,87553,91970,11109,11370,11989,7126,4217,1894,1469,...,2453,5176,8222,46418,30765,33869,3987,3494,8204,55072
2,27546,14649,12897,1669,1837,1540,1087,656,245,362,...,591,1961,2672,7761,3900,2630,1523,1044,1329,5693
3,22746,11848,10898,1307,1689,1439,928,499,204,222,...,56,1531,1806,7401,3053,1500,460,623,1157,4985
4,57140,28357,28783,3601,3721,4251,2487,1425,688,689,...,736,3884,5606,16920,7716,4347,1560,1348,2897,15149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,59995,28898,31097,3800,4158,4695,2814,1803,962,979,...,738,9079,3784,12192,6869,6744,6011,2377,3300,6146
3217,9318,4625,4693,644,519,695,398,240,191,87,...,755,1368,948,2155,853,844,981,426,805,987
3218,26290,12973,13317,1786,1972,2157,1483,1008,584,355,...,781,4067,1400,5758,2318,2239,2627,1070,1369,2598
3219,38138,18609,19529,2452,2577,2876,1796,1179,723,524,...,8680,6089,2806,7969,4292,3386,3644,1609,2428,4094


In [24]:
nhgis_cleaned[cols_2011]

,population_2011,male_2011,female_2011,under_5__years_2011,5_to_9_years_2011,10_to_14_years_2011,15_to_17_years_2011,18_to_19_years_2011,20_years_2011,21_years_2011,...,Two_or_More_Races_2011,Less_than_9th_grade_2011,9th_to_11th_grade_2011,12th_grade_or_high_school_graduate_2011,1_to_3_years_of_college_2011,4_or_more_years_of_college_2011,"less_than_10,000_2011","10,000_to_14,999_2011","15,000_to_24,999_2011","25,000_or_more_2011"
0,54907,26793,28114,3489,3905,4398,2600,1583,730,561,...,755,1785,2722,15533,8184,7472,1264,1099,1805,15903
1,187114,91413,95701,11284,12386,11546,7436,4365,1874,2084,...,2697,4891,7560,48083,33442,36087,4776,3571,8996,55940
2,27321,14634,12687,1591,1730,1577,991,607,365,246,...,489,1827,2708,7752,4156,2545,1326,1063,1423,5388
3,22754,12343,10411,1290,1472,1359,938,703,211,446,...,218,1339,1895,7045,3210,1851,701,472,1379,4539
4,57623,28604,29019,3631,3693,4175,2499,1473,616,656,...,979,3010,5054,17150,8935,4697,1664,1159,2753,15532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,58782,28315,30467,3472,3913,4318,2680,1744,871,1047,...,689,8063,3816,13241,6475,7025,5779,2229,3367,6009
3217,9273,4695,4578,634,509,626,386,295,186,37,...,666,1418,829,2454,878,664,695,396,741,953
3218,25618,12581,13037,1623,1733,2040,1402,953,543,442,...,4270,3744,1381,5884,2246,2566,2482,1099,1477,2765
3219,37404,18223,19181,2237,2303,2797,1709,1128,678,568,...,5407,5940,2486,8124,4775,3266,3744,1645,2201,4318


In [25]:
# create the 2009 dataset by combining:
# (1) geographic identifier columns, and
# (2) all demographic variables that are from 2009 ACS
# 'axis=1' ensures we concatenate column-wise
nhgis_2009 = pd.concat(
    [nhgis_cleaned[geo_cols], nhgis_cleaned[cols_2009]], axis=1
).copy()

# add a YEAR column so that later we can use it in groupby
nhgis_2009["YEAR"] = 2009

In [26]:
# build a rename dictionary to remove the "_2009" suffix from all related variable
rename_2009 = {}
for col in cols_2009:
    new_name = col.replace("_2009", "")
    rename_2009[col] = new_name
nhgis_2009 = nhgis_2009.rename(columns=rename_2009)
nhgis_2009.columns  # view column names

Index(['GISJOIN', 'STATE', 'COUNTY', 'COUNTYFP', 'population', 'male',
       'female', 'under_5__years', '5_to_9_years', '10_to_14_years',
       '15_to_17_years', '18_to_19_years', '20_years', '21_years',
       '22_to_24_years', '25_to_29_years', '30_to_34_years', '35_to_44_years',
       '45_to_54_years', '55_to_59_years', '60_to_61_years', '62_to_64_years',
       '65_to_74_years', '75_to_84_years', '85_years_and_over', 'White',
       'Black_or_African_American', 'American_Indian_and_Alaska_Native',
       'Asian_and_Pacific_Islander_and_Other_Race', 'Two_or_More_Races',
       'Less_than_9th_grade', '9th_to_11th_grade',
       '12th_grade_or_high_school_graduate', '1_to_3_years_of_college',
       '4_or_more_years_of_college', 'less_than_10,000', '10,000_to_14,999',
       '15,000_to_24,999', '25,000_or_more', 'YEAR'],
      dtype='object')

In [27]:
# create the 2011 dataset by combining:
# (1) geographic identifier columns, and
# (2) all demographic variables that are from 2011 ACS
# 'axis=1' ensures we concatenate column-wise
nhgis_2011 = pd.concat(
    [nhgis_cleaned[geo_cols], nhgis_cleaned[cols_2011]], axis=1
).copy()

# add a YEAR column so that later we can use it in groupby
nhgis_2011["YEAR"] = 2011

In [28]:
# build a rename dictionary to remove the "_2011" suffix from all related variable
rename_2011 = {}
for col in cols_2011:
    new_name = col.replace("_2011", "")
    rename_2011[col] = new_name
nhgis_2011 = nhgis_2011.rename(columns=rename_2011)
nhgis_2011.columns  # view column names

Index(['GISJOIN', 'STATE', 'COUNTY', 'COUNTYFP', 'population', 'male',
       'female', 'under_5__years', '5_to_9_years', '10_to_14_years',
       '15_to_17_years', '18_to_19_years', '20_years', '21_years',
       '22_to_24_years', '25_to_29_years', '30_to_34_years', '35_to_44_years',
       '45_to_54_years', '55_to_59_years', '60_to_61_years', '62_to_64_years',
       '65_to_74_years', '75_to_84_years', '85_years_and_over', 'White',
       'Black_or_African_American', 'American_Indian_and_Alaska_Native',
       'Asian_and_Pacific_Islander_and_Other_Race', 'Two_or_More_Races',
       'Less_than_9th_grade', '9th_to_11th_grade',
       '12th_grade_or_high_school_graduate', '1_to_3_years_of_college',
       '4_or_more_years_of_college', 'less_than_10,000', '10,000_to_14,999',
       '15,000_to_24,999', '25,000_or_more', 'YEAR'],
      dtype='object')

In [29]:
# concatenate the 2009 and 2011 datasets into a single panel
# 'ignore_index=True' resets the row index after concatenating the two datasets
nhgis_panel = pd.concat([nhgis_2009, nhgis_2011], ignore_index=True)
nhgis_panel

,GISJOIN,STATE,COUNTY,COUNTYFP,population,male,female,under_5__years,5_to_9_years,10_to_14_years,...,Less_than_9th_grade,9th_to_11th_grade,12th_grade_or_high_school_graduate,1_to_3_years_of_college,4_or_more_years_of_college,"less_than_10,000","10,000_to_14,999","15,000_to_24,999","25,000_or_more",YEAR
0,G0100010,Alabama,Autauga County,1,53944,26174,27770,3568,4095,4209,...,1551,2555,14997,8000,7478,1304,843,1757,16094,2009
1,G0100030,Alabama,Baldwin County,3,179523,87553,91970,11109,11370,11989,...,5176,8222,46418,30765,33869,3987,3494,8204,55072,2009
2,G0100050,Alabama,Barbour County,5,27546,14649,12897,1669,1837,1540,...,1961,2672,7761,3900,2630,1523,1044,1329,5693,2009
3,G0100070,Alabama,Bibb County,7,22746,11848,10898,1307,1689,1439,...,1531,1806,7401,3053,1500,460,623,1157,4985,2009
4,G0100090,Alabama,Blount County,9,57140,28357,28783,3601,3721,4251,...,3884,5606,16920,7716,4347,1560,1348,2897,15149,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6437,G7201450,Puerto Rico,Vega Baja Municipio,145,58782,28315,30467,3472,3913,4318,...,8063,3816,13241,6475,7025,5779,2229,3367,6009,2011
6438,G7201470,Puerto Rico,Vieques Municipio,147,9273,4695,4578,634,509,626,...,1418,829,2454,878,664,695,396,741,953,2011
6439,G7201490,Puerto Rico,Villalba Municipio,149,25618,12581,13037,1623,1733,2040,...,3744,1381,5884,2246,2566,2482,1099,1477,2765,2011
6440,G7201510,Puerto Rico,Yabucoa Municipio,151,37404,18223,19181,2237,2303,2797,...,5940,2486,8124,4775,3266,3744,1645,2201,4318,2011


In [30]:
# aggregate the data to ensure one row per county-year
nhgis_final = nhgis_panel.groupby(
    ["GISJOIN", "STATE", "COUNTY", "COUNTYFP", "YEAR"]
).sum()
nhgis_final

population   male  \
GISJOIN  STATE       COUNTY             COUNTYFP YEAR                      
G0100010 Alabama     Autauga County     1        2009       53944  26174   
                                                 2011       54907  26793   
G0100030 Alabama     Baldwin County     3        2009      179523  87553   
                                                 2011      187114  91413   
G0100050 Alabama     Barbour County     5        2009       27546  14649   
...                                                           ...    ...   
G7201490 Puerto Rico Villalba Municipio 149      2011       25618  12581   
G7201510 Puerto Rico Yabucoa Municipio  151      2009       38138  18609   
                                                 2011       37404  18223   
G7201530 Puerto Rico Yauco Municipio    153      2009       42516  20598   
                                                 2011       41194  19911   

                                                       female  under_5__years  \
GISJOIN  STATE       COUNTY             COUNTYFP YEAR                           
G0100010 Alabama     Autauga County     1        2009   27770            3568   
                                                 2011   28114            3489   
G0100030 Alabama     Baldwin County     3        2009   91970           11109   
                                                 2011   95701           11284   
G0100050 Alabama     Barbour County     5        2009   12897            1669   
...                                                       ...             ...   
G7201490 Puerto Rico Villalba Municipio 149      2011   13037            1623   
G7201510 Puerto Rico Yabucoa Municipio  151      2009   19529            2452   
                                                 2011   19181            2237   
G7201530 Puerto Rico Yauco Municipio    153      2009   21918            2554   
                                                 2011   21283            2349   

                                                       5_to_9_years  \
GISJOIN  STATE       COUNTY             COUNTYFP YEAR                 
G0100010 Alabama     Autauga County     1        2009          4095   
                                                 2011          3905   
G0100030 Alabama     Baldwin County     3        2009         11370   
                                                 2011         12386   
G0100050 Alabama     Barbour County     5        2009          1837   
...                                                             ...   
G7201490 Puerto Rico Villalba Municipio 149      2011          1733   
G7201510 Puerto Rico Yabucoa Municipio  151      2009          2577   
                                                 2011          2303   
G7201530 Puerto Rico Yauco Municipio    153      2009          2661   
                                                 2011          2388   

                                                       10_to_14_years  \
GISJOIN  STATE       COUNTY             COUNTYFP YEAR                   
G0100010 Alabama     Autauga County     1        2009            4209   
                                                 2011            4398   
G0100030 Alabama     Baldwin County     3        2009           11989   
                                                 2011           11546   
G0100050 Alabama     Barbour County     5        2009            1540   
...                                                               ...   
G7201490 Puerto Rico Villalba Municipio 149      2011            2040   
G7201510 Puerto Rico Yabucoa Municipio  151      2009            2876   
                                                 2011            2797   
G7201530 Puerto Rico Yauco Municipio    153      2009            3305   
                                                 2011            3068   

                                                       15_to_17_years  \
GISJOIN  STATE       COUNTY             COUNTYFP YEAR            

In [31]:
# read nhgis dataset
nh = nhgis_panel.copy()
nh.columns

Index(['GISJOIN', 'STATE', 'COUNTY', 'COUNTYFP', 'population', 'male',
       'female', 'under_5__years', '5_to_9_years', '10_to_14_years',
       '15_to_17_years', '18_to_19_years', '20_years', '21_years',
       '22_to_24_years', '25_to_29_years', '30_to_34_years', '35_to_44_years',
       '45_to_54_years', '55_to_59_years', '60_to_61_years', '62_to_64_years',
       '65_to_74_years', '75_to_84_years', '85_years_and_over', 'White',
       'Black_or_African_American', 'American_Indian_and_Alaska_Native',
       'Asian_and_Pacific_Islander_and_Other_Race', 'Two_or_More_Races',
       'Less_than_9th_grade', '9th_to_11th_grade',
       '12th_grade_or_high_school_graduate', '1_to_3_years_of_college',
       '4_or_more_years_of_college', 'less_than_10,000', '10,000_to_14,999',
       '15,000_to_24,999', '25,000_or_more', 'YEAR'],
      dtype='object')

In [32]:
# 4 states
tar_states = ["California", "Florida", "New Jersey", "Oregon"]
nh_1 = nh[nh["STATE"].isin(tar_states)]

In [33]:
# Adjust GISJOIN into County Code
nh_1["STATEFP"] = nh_1["STATE"].map(
    {"California": "06", "Florida": "12", "New Jersey": "34", "Oregon": "41"}
)

nh_1["County Code"] = nh_1["STATEFP"] + nh_1["COUNTYFP"].astype(str).str.zfill(3)

> ### 1 tip:
> this analysis just operationalize the policy change as having taken place in February 2010.

In [34]:
# mapping the year_group to year
def map_year_to_group(y):
    if 2007 <= y <= 2009:
        return 2009  # for year 2009 in nh dataset
    elif 2010 <= y <= 2012:
        return 2011  # for year 2011 in nh dataset
    else:
        return pd.NA

In [35]:
# add 'year group' to all three dataframes
death_overdose["year_group"] = death_overdose["year"].apply(map_year_to_group)
opioids_agg["year_group"] = opioids_agg["year"].apply(map_year_to_group)
nh_1["year_group"] = nh_1["YEAR"]

In [36]:
# rename countyfips to match the county identifier used in the other datasets
opioids_agg = opioids_agg.rename(columns={"countyfips": "County Code"})

In [37]:
# merge death + opioid
df_all = death_overdose.merge(
    opioids_agg, on=["County Code", "year", "year_group"], how="outer"
)
df_all.head(10)

,County Code,year,overdose_deaths,year_group,total_grams,total_mme
0,06001,2007,163.0,2009,825601.233034,1052558.500894
1,06001,2008,135.0,2009,847083.303522,1117798.930286
2,06001,2009,119.0,2009,875071.886798,1199622.098446
3,06001,2010,106.0,2011,870211.603741,1247010.639905
4,06001,2011,139.0,2011,896433.100448,1296731.400855
5,06001,2012,115.0,2011,900341.807573,1313075.53561
6,06005,2007,NaN,2009,28486.513615,50147.191799
7,06005,2008,NaN,2009,31402.138263,56498.072585
8,06005,2009,13.0,2009,34543.746528,63986.757334
9,06005,2010,NaN,2011,34834.035017,66070.709978


In [38]:
# add NHGIS data
df_all = df_all.merge(nh_1, on=["County Code", "year_group"], how="left")
df_all.head(10)

,County Code,year,overdose_deaths,year_group,total_grams,total_mme,GISJOIN,STATE,COUNTY,COUNTYFP,...,9th_to_11th_grade,12th_grade_or_high_school_graduate,1_to_3_years_of_college,4_or_more_years_of_college,"less_than_10,000","10,000_to_14,999","15,000_to_24,999","25,000_or_more",YEAR,STATEFP
0,06001,2007,163.0,2009,825601.233034,1052558.500894,G0600010,California,Alameda County,1,...,45786,265831,210108,411652,27989,25930,43910,438331,2009,06
1,06001,2008,135.0,2009,847083.303522,1117798.930286,G0600010,California,Alameda County,1,...,45786,265831,210108,411652,27989,25930,43910,438331,2009,06
2,06001,2009,119.0,2009,875071.886798,1199622.098446,G0600010,California,Alameda County,1,...,45786,265831,210108,411652,27989,25930,43910,438331,2009,06
3,06001,2010,106.0,2011,870211.603741,1247010.639905,G0600010,California,Alameda County,1,...,43297,268460,218608,436208,29125,26168,43635,446143,2011,06
4,06001,2011,139.0,2011,896433.100448,1296731.400855,G0600010,California,Alameda County,1,...,43297,268460,218608,436208,29125,26168,43635,446143,2011,06
5,06001,2012,115.0,2011,900341.807573,1313075.53561,G0600010,California,Alameda County,1,...,43297,268460,218608,436208,29125,26168,43635,446143,2011,06
6,06005,2007,NaN,2009,28486.513615,50147.191799,G0600050,California,Amador County,5,...,1997,11891,8750,5439,658,622,1524,11479,2009,06
7,06005,2008,NaN,2009,31402.138263,56498.072585,G0600050,California,Amador County,5,...,1997,11891,8750,5439,658,622,1524,11479,2009,06
8,06005,2009,13.0,2009,34543.746528,63986.757334,G0600050,California,Amador County,5,...,1997,11891,8750,5439,658,622,1524,11479,2009,06
9,06005,2010,NaN,2011,34834.035017,66070.709978,G0600050,California,Amador County,5,...,1923,11266,9086,5575,844,729,1531,11158,2011,06


In [39]:
# compute normalized rate
df_all["overdose_rate_per_100k"] = (
    df_all["overdose_deaths"] / df_all["population"] * 100000
)

df_all["mme_per_capita"] = df_all["total_mme"] / df_all["population"]
df_all["COUNTY"].value_counts()

COUNTY
Union County       18
Lake County        18
Jackson County     12
Orange County      12
Columbia County    12
                   ..
Bay County          6
Bradford County     6
Brevard County      6
Yamhill County      6
Glades County       4
Name: count, Length: 167, dtype: int64

In [40]:
# remove counties with population less than 350,000
df_all_big = df_all[df_all["population"] >= 350000]
df_all_big

,County Code,year,overdose_deaths,year_group,total_grams,total_mme,GISJOIN,STATE,COUNTY,COUNTYFP,...,1_to_3_years_of_college,4_or_more_years_of_college,"less_than_10,000","10,000_to_14,999","15,000_to_24,999","25,000_or_more",YEAR,STATEFP,overdose_rate_per_100k,mme_per_capita
0,06001,2007,163.0,2009,825601.233034,1052558.500894,G0600010,California,Alameda County,1,...,210108,411652,27989,25930,43910,438331,2009,06,10.903914,0.704111
1,06001,2008,135.0,2009,847083.303522,1117798.930286,G0600010,California,Alameda County,1,...,210108,411652,27989,25930,43910,438331,2009,06,9.030849,0.747754
2,06001,2009,119.0,2009,875071.886798,1199622.098446,G0600010,California,Alameda County,1,...,210108,411652,27989,25930,43910,438331,2009,06,7.960526,0.802489
3,06001,2010,106.0,2011,870211.603741,1247010.639905,G0600010,California,Alameda County,1,...,218608,436208,29125,26168,43635,446143,2011,06,6.904422,0.812254
4,06001,2011,139.0,2011,896433.100448,1296731.400855,G0600010,California,Alameda County,1,...,218608,436208,29125,26168,43635,446143,2011,06,9.053912,0.84464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,41067,2008,21.0,2009,247109.78572,453028.963358,G4100670,Oregon,Washington County,67,...,87609,136501,8660,7520,15981,166432,2009,41,4.005531,0.864106
1062,41067,2009,32.0,2009,250921.039374,466531.974526,G4100670,Oregon,Washington County,67,...,87609,136501,8660,7520,15981,166432,2009,41,6.103667,0.889861
1063,41067,2010,17.0,2011,258714.909532,499542.2896,G4100670,Oregon,Washington County,67,...,90452,141296,9427,7345,16733,168266,2011,41,3.150435,0.92575
1064,41067,2011,31.0,2011,258331.473011,495044.591781,G4100670,Oregon,Washington County,67,...,90452,141296,9427,7345,16733,168266,2011,41,5.744911,0.917415
